# Home assignment 2

You should work on the assignement in groups of 2 participants. 

Upload your solution as a jupyter notebook to L2P by 26th of June 23:59h. (The deadline is strict)

Do not forget to specify the names of all contributing students in the jupyter notebook.

You should add comments to your code where necessary and print the relevant results.

# 1. Dynamic PageRank
Consider a random walk setting where the transistion matrix changes over time. At any point of time the probability of a random surfer to jump to a linked page is proportional to the number of previous visits. To start with all the pages are equally likely to be chosen but as the walk continues and the nodes are visited the transition probability changes as proportional to number of previous visits. For example let a page 'a' is linked to pages 'b', 'c' and 'd'. The random surfer currently resides at 'a' and the pages 'b', 'c' and 'd' have already been visited 5, 3 and 2 times respectively. The transition probability would be 0.5, 0.3 and 0.2 respectively. As a new node is viited the probabilities change. The random surfer continues to surf with probability 0.8. Generate 100 random walks and rank the nodes based on the frequency of visit. The random walk should be performed on a drected Erdos-Renyi graph with number of nodes n=200 and probability of edge creation p = 0.4. 

Hint: Use networkx library for generating graph.
    

In [12]:
import networkx as nx
import random

n = 200
p = 0.4
G = nx.erdos_renyi_graph(n, p, directed = True)
pr = nx.pagerank(G, alpha=0.8)
print('Pagerank:', pr)
nodes = list(G.nodes())
visit_freq = {}
for node in nodes:
    visit_freq[node] = 0

p_restart = 1 - 0.8
random.shuffle(nodes)
walks = 100
for i in range(walks):
    random.shuffle(nodes)
    current_node = nodes[0]
    visit_freq[current_node] += 1
    
    while random.uniform(0, 1) > p_restart: # with 0.8 probability continue to do this walk
        neighbors = list(G.neighbors(current_node))
        for neighbor in neighbors:
            random.shuffle(neighbors) # randomly choosing a neighbor
            current_node = neighbors[0]
            visit_freq[current_node] += 1
print('visited frequencies:', visit_freq)


{0: 0.00520391813792562, 1: 0.004995726769684962, 2: 0.004760431008129827, 3: 0.004696097775524438, 4: 0.005341112679087715, 5: 0.005003241540565149, 6: 0.004750131802252447, 7: 0.0053703082977794026, 8: 0.004663860597871898, 9: 0.004787749777256244, 10: 0.0054408901028099215, 11: 0.005095605526170962, 12: 0.004481346164462374, 13: 0.0049058426927969774, 14: 0.0044970445237254245, 15: 0.005035227353009484, 16: 0.004905679882560042, 17: 0.004673049408587545, 18: 0.004786218330503262, 19: 0.004531175151362466, 20: 0.005142083393242547, 21: 0.005250591024378294, 22: 0.00534520370325815, 23: 0.004948428423590077, 24: 0.004812584816864742, 25: 0.005582271857702124, 26: 0.0048786452915167, 27: 0.004780982523933035, 28: 0.004743117272051064, 29: 0.004500561674296368, 30: 0.005807226222913962, 31: 0.004880277637705891, 32: 0.0050185633884252435, 33: 0.004815925582718899, 34: 0.004979256234973359, 35: 0.005201911986157827, 36: 0.0047820643659054135, 37: 0.004958638308205054, 38: 0.0048000806938

# 2. Recommendation
a. Compare the recommendation algorithms (SVD, NMF, Baseline, k-NN and Random) available in surprise package on movielens dataset in terms of RMSE and MAE.

b. Consider the movielens dataset and divide it into (i) training set with 50% of the data (train the algorithms on this part) and (ii) 25% validation set and (iii) test set with the rest. Estimate the ratings of the test set using the algorithms (same as in a) provided by the package on the training set. Your final rating should be weighted average of the ratings predicted by the algorithms. The weights should be learnt on the validation set. Performance should be measured in terms of RMSE.

Hint: Use grid search/step-wise update like SGD for learning the weights

In [8]:
from surprise import SVD, NMF, BaselineOnly, KNNBasic, NormalPredictor
from surprise import Dataset, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise.model_selection import train_test_split 
import pandas as pd
import matplotlib as plt
import numpy as np

# x: input examples, y: y value for each x, w: weights matrix
# l_rate: learning rate, m: number of input examples
# epochs: the number of iterations
def gradient_descent(x, y, w, l_rate, m, epochs):
    x_trans = x.transpose()
    for i in range(0, epochs):
        hypothesis = np.dot(x, w)
        loss = hypothesis - y
        # avg cost
        cost = np.sum(loss ** 2) / (2 * m)
        #print("Iteration %d | Cost: %f" % (i, cost))
        # avg gradient
        gradient = np.dot(x_trans, loss) / m
        # update weight
        w = w - l_rate * gradient
    return w

def final_predicted_rating(x, w):
    hypothesis = np.dot(x, w)
    return hypothesis

def build_examples(dataset):    
    predictions = []
    # stroing prediction of each model respectively.
    for model in algo_names:
        prediction = models[model].test(dataset)
        p_array = []
        for id, p in enumerate(prediction):
            # user: 311     item: 15         r_ui = 5.00   est = 4.07
            # p[0] -- user, p[1] --- item
            p_array.append((p[0], p[1], p.est))
        predictions.append(p_array)

    # predicted values of each algorithm
    est_values = []
    for idx in range(len(predictions[0])):
        values = []
        for idy in range(len(predictions)):
            # predictions[idy][idx][2]: predicted value of algorithm with index "idx" for one item.
            values.append(predictions[idy][idx][2])
        est_values.append(values)
    est_values = np.array(est_values)

    # all original ratings
    rating_values = []
    for v in dataset:
        rating_values.append(v[2])
    rating_values = np.array(rating_values)
    return est_values, rating_values


algorithms = [SVD, NMF, BaselineOnly, KNNBasic, NormalPredictor]

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')
trainset, restset = train_test_split(data, test_size=0.5) # 50% training set, 50% for validation and test
validationset = restset[:round(len(restset)/2)]
testset = restset[round(len(restset)/2):]
# user film rating ('506', '731', 4.0)

algo_names = ['SVD', 'NMF', 'Baseline', 'k-NN', 'Random']
models = {}
rmse_mae = []
accuracies = {}
# Algorithms.
for index, algo in enumerate(algorithms):
    # for subtask a), Run 5-fold cross-validation and print results. 
    out = cross_validate(algo(), data, measures=['RMSE', 'MAE'], cv=3)
    #rmse_mae.append( ['{:.3f}'.format(np.mean(out['test_rmse'])), '{:.3f}'.format(np.mean(out['test_mae']))] )
    print(algo_names[index] + ':', 'RMSE {:.3f}'.format(np.mean(out['test_rmse'])), 'MAE {:.3f}'.format(np.mean(out['test_mae'])))
    
    # subtask b)
    algo = algo()
    algo.fit(trainset)
    models[algo_names[index]] = algo


v_est_values, v_rating_values = build_examples(validationset)
t_est_values, t_rating_values = build_examples(testset)
# weights
weights = np.ones(len(algorithms))
learning_rate = 0.0005
epochs = 10000
v_m, v_n = np.shape(est_values)
t_m, t_n = np.shape(t_est_values)
# learning weight on validation set
weights = gradient_descent(v_est_values, v_rating_values, weights, learning_rate, v_m, epochs)
print('weights:', weights)

# final ratings
final_ratings = final_predicted_rating(t_est_values, weights)

# performance
final_rmse = np.sqrt(np.sum((final_ratings - t_rating_values) ** 2) / (2 * t_m))
print('Perfomance of combiniation of five algorithms, final RMSE:'final_rmse)
    


SVD: 0.945 0.747
NMF: 0.975 0.766
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Baseline: 0.948 0.752
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
k-NN: 0.987 0.780
Computing the msd similarity matrix...
Done computing similarity matrix.
Random: 1.519 1.219
0.6672805089811458


# 3. Hidden Markov model
Consider the HMM package https://hmmlearn.readthedocs.io/en/latest/

a. Generate sequences with multinomial HMM (2 symbols and 4 hidden states) and given parameters. Start probability - {0.4,0.2,0.1,0.3}, Transition matrix - {{0.2,0.3,0.1,0.4},{0.3,0.3,0.2,0.2},{0.4,0.2,0.3,0.1},{0.2,0.3,0.1,0.4}}, Emission probability - {{0.2,0.8},{0.1,0.9},{0.5,0.5},{0.6,0.4}}.


b. Consider a sequence - {1 0 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 1 0 1}. Fit a multinomial HMM considering 4 states and obtain hidden state which is most likely to have generated the symbol


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn.hmm import MultinomialHMM

startprob = np.array([0.4,0.2,0.1,0.3])
transition_matrix = np.array([[0.2,0.3,0.1,0.4],
                             [0.3,0.3,0.2,0.2],
                             [0.3,0.3,0.2,0.2],
                             [0.4,0.2,0.3,0.1],
                             [0.2,0.3,0.1,0.4]])
emissionprob = np.array([[0.2,0.8],
                        [0.1,0.9],
                        [0.5,0.5],
                        [0.6,0.4]])
seq = [[1], [0], [0], [0], [1], [1], [1], [1], [0], [1], [0], [1], [0], [1], [0], [1], [1], [1], [0], [0], [0], [1], [1], [0], [1], [0], [0], [1], [1], [0], [1]]

model = MultinomialHMM(n_components=4)
model.transmat_ = transition_matrix
model.startprob_ = startprob
model.emissionprob_ = emissionprob
X, Z = model.sample(100) # a
model.fit(seq) # b
# print(X)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=4,
        n_iter=10, params='ste',
        random_state=<mtrand.RandomState object at 0x1092f74c8>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False)

# 4. PrefixSpan 
Implement the Prefix algorithm for Sequential Pattern Mining.

Input of the algorithm :  

	A sequence database S, 
and the minimum support threshold min_support.

 - Output of the algorithm:  
	 The complete set of sequential patterns.
 - Subroutine: PrefixSpan(α, L, S|α).
 
Parameters: 
- α: sequential pattern, 
- L: the length of α; 
- S|α: : the α-projected database, if α ≠<>; otherwise;  the sequence database S.

Call PrefixSpan(<>,0,S).

1. Scan S|α once, 
find the set of frequent items b such that:
 
 b can be assembled to the last element of α to form a sequential pattern; or
 \<b\> can be appended to α to form a sequential pattern.
 
2. For each frequent item b:
- append it to α to form a sequential pattern α’ and output α’;
- output α’;

3. For each α’:
 - construct α’-projected database S|α’ and call PrefixSpan(α’, L+1, S|α’).

In [217]:
import copy
# [a[b,c], [bcd[ab]]] three-dimensional
class PrefixSpan:
    
    def __init__(self, sequences, min_support=0.5):
        
        self.min_support = round(min_support * len(sequences))
        self.PLACE_HOLDER = '_'
        self.s_patterns = None
        self.s_patterns = self.prefix_span(self.SequencePattern([], None, self.PLACE_HOLDER), 
                                           sequences, 0, self.min_support, None)
        
    def get_all_patterns(self):
        return self.s_patterns
        
    def prefix_span(self, seq_pattern, sequences, pattern_length, support, test):
        patterns = list()
 
        if len(sequences) < support:
            return patterns
        
        f_items = self.frequent_item(sequences, seq_pattern, support)
        
        for f_i in f_items:
            #print('current frequent item:', f_i.sequence, 'pattern seq:', id(seq_pattern.sequence))
            p = self.SequencePattern(seq_pattern.sequence, seq_pattern.freq, self.PLACE_HOLDER)
            
            
            p.append(f_i)
            patterns.append(p)
            tmp = copy.deepcopy(p) # workaround
            p_db = self.build_projected_database(sequences, p)
            p_patterns = self.prefix_span(tmp, p_db, 0, support, patterns)
            patterns.extend(p_patterns)
            
        return patterns
        
    
    def frequent_item(self, seqs, pattern, support):
        items = {} # normal patterns <b> + <c> => <bc>
        multi_items = {} # special patterns <(ab> + <c)> => (<abc)>
        f_item_list = []
        
        if seqs is None or len(seqs) == 0:
            return []
        
        if len(pattern.sequence) != 0:
            last_el = pattern.sequence[-1]
        else:
            last_el = []
            
        for seq in seqs:
            # 1. Pattern
            # current pattern: [[...], [a,b]], database: [[a,b,c],[...]]
            # attaching item c to the element [a,b] of the pattern, getting the new pattern [a,b,c]
            if self.PLACE_HOLDER != seq[0][0]:
                is_prefix = True
                for item in last_el: #test last element of the given pattern occurs 
                    if item not in seq[0]:
                        is_prefix = False
                        break
                if is_prefix and len(last_el) > 0:
                    last_item = last_el[-1]
                    item_index = seq[0].index(last_item)
                    if item_index < len(seq[0]):
                        for item in seq[0][item_index+1:]:
                            if item not in multi_items:
                                multi_items[item] = 1
                            else:
                                multi_items[item] += 1
                                
            # 2. Pattern
            # current pattern: [[...], [a,b]], database: [[_,c],[...]]
            # attaching item c to the element [a,b] of the pattern, getting the new pattern [a,b,c]
            elif self.PLACE_HOLDER == seq[0][0]:
                for item in seq[0][1:]:
                    if item in multi_items:
                        multi_items[item] += 1
                    else:
                        multi_items[item] = 1
                
                seq = seq[1:] # skipping the first element seq[0], because we have collected all pattern from seq[0]
            # 3. Pattern
            recorded = []
            for el in seq:
                for item in el:
                    if item not in recorded:
                        recorded.append(item)
                        if item not in items:
                            items[item] = 1
                        else:
                            items[item] += 1
            
        # applying each character in multi_items and items having minimal support to build up all new patterns
        f_item_list.extend([self.SequencePattern([[self.PLACE_HOLDER, key]], value, self.PLACE_HOLDER) 
                          for key, value in multi_items.items() if value >= support])
        f_item_list.extend([self.SequencePattern([[key]], value, self.PLACE_HOLDER) 
                          for key, value in items.items() if value >= support])
        #print('multi_items:', multi_items)
#         for f in f_item_list:
#             print('f_item_list:', f.sequence)
        return f_item_list
            
            
    class SequencePattern:
        def __init__(self, item_set, item_freq, place_holder):
            self.sequence = []
            self.place_holder = place_holder
            self.freq = None # applying for sorting
            for item in item_set:
                self.sequence.append(item)
            self.freq = item_freq
        
        def append(self, pattern):
            if pattern.sequence[0][0] == self.place_holder:
                self.sequence[-1].extend(pattern.sequence[0][1:])
                self.sequence.extend(pattern.sequence[1:])
            else:
                self.sequence.extend(pattern.sequence)
                
                if self.freq is None:
                    self.freq = pattern.freq
            self.freq = min(self.freq, pattern.freq)
                 

    def build_projected_database(self, seqs, pattern):
        projected_seqs = []
        
        if pattern is None or seqs is None:
            return []
        
        # pattern.sequence = [[1],[2],[3,4]] : 12(34)
        last_el = pattern.sequence[-1]
        last_item = last_el[-1]
        for seq in seqs:
            p_seqs = []
            for el in seq:
                is_prefix = False
                if el[0] == self.PLACE_HOLDER: # el=[_ab] or [a_b]? the order within an element no matters
                    if last_item in el and len(pattern.sequence[-1]) > 1: # since '_' and another char in el so that at least 2 chars
                        is_prefix = True
                else:
                    is_prefix = True
                    for item in last_el:
                        if item not in el:
                            is_prefix = False
                            break
                
                if is_prefix:
                    el_index = seq.index(el)
                    item_index = el.index(last_item)
                    if item_index == len(el) - 1:
                        p_seqs = seq[el_index + 1:]
                    else:
                        p_seqs = seq[el_index:]
                        tmp_el = el[item_index:]
                        tmp_el[0] = self.PLACE_HOLDER
                        p_seqs[0] = tmp_el
                        
                        
                    break # find the corresponding projected database of given pattern in current sequence
                
            if len(p_seqs) != 0:
                projected_seqs.append(p_seqs)
        return projected_seqs
        
if __name__ == "__main__":

    sequences = [
        [[1,2],[3]],
        [[1],[3,2],[1,2]],
        [[1,2],[5]],
        [[6]],
    ]

    model = PrefixSpan(sequences, min_support=0.5)
    result = model.get_all_patterns()
    print('Result:')
    for fs in result:
        #print('{}, {}'.format(fs.sequence,fs.freq))
        print('{}'.format(fs.sequence))

        


Result:
[[1]]
[[1, 2]]
[[1, 2], [3]]
[[2]]
[[3]]
